In [ ]:
# %matplotlib ipympl

import os
import glob
import torch
import tqdm
import time
import csv
from torch import nn
from torch.utils.data import DataLoader, ConcatDataset, random_split
from learning.model.actionvalue import DWActionValueModel
from learning.data.process_utils import move_to_device
from learning.data.av_utils import (
    extract_input, extract_output_target, collate_av_data
)
from learning.data.av_dataset import ActionValueDataset

In [3]:
history_folder = "server/history/"
history_files = sorted(glob.glob(os.path.join(history_folder, "history_*.json")))

datasets = [ActionValueDataset(f) for f in history_files]
concatenated_dataset = ConcatDataset(datasets)

train_ds, val_ds, test_ds = random_split(
    concatenated_dataset,
    [0.8, 0.1, 0.1],
)

In [4]:
# action_values = torch.stack([
#     d["action_value"] 
#     for d in train_ds
# ])
# mean = action_values.mean()
# std = action_values.std()
# mean, std
# (tensor(-820.0759), tensor(1868.9108))

In [ ]:
mean_val = torch.tensor(-820)
std_val = torch.tensor(1870)

In [6]:

# Set a fixed seed for reproducibility
seed = 42
torch.manual_seed(seed)
generator = torch.Generator().manual_seed(seed)

train_loader = DataLoader(train_ds, 16000, pin_memory=True, shuffle=True, 
                         collate_fn=collate_av_data, generator=generator)
val_loader = DataLoader(val_ds, 20000, pin_memory=True, shuffle=True, 
                       collate_fn=collate_av_data, generator=generator)

print("train_ds", len(train_ds))
print("val_ds", len(val_ds))
print("test_ds", len(test_ds))

train_ds 324610
val_ds 40576
test_ds 40576


In [7]:
model_state = None
optim_state = None
latest_epoch = -1

model_name = "actionvalue_with_freeze"
models_dir = f"learning/{model_name}_checkpoints" 

os.makedirs(models_dir, exist_ok=True)

# Find the latest checkpoint file (with highest epoch number)
checkpoint_files = glob.glob(os.path.join(models_dir, f"{model_name}_*.pt"))
if checkpoint_files:
    # Extract epoch numbers from filenames
    epoch_nums = [int(f.split("_")[-1].split(".")[0]) for f in checkpoint_files]
    latest_epoch = max(epoch_nums)
    latest_checkpoint = os.path.join(models_dir, f"{model_name}_{latest_epoch:06}.pt")
    print(f"Loading latest checkpoint: {latest_checkpoint} (epoch {latest_epoch})")
    checkpoint = torch.load(latest_checkpoint, map_location=torch.device('cpu'))
    model_state = checkpoint["model_state"]
    optim_state = checkpoint["optim_state"]
else:
    print("No valid checkpoint files found")
    
start_epoch = latest_epoch + 1

No valid checkpoint files found


In [8]:
action_value_model = DWActionValueModel()
num_params = sum(p.numel() for p in action_value_model.parameters() if p.requires_grad)
print(f"Model created. Total trainable parameters: {num_params:,}")

Model created. Total trainable parameters: 277


In [9]:
init_from_old_model = (model_state is None)

if init_from_old_model:
    print("⚠️⚠️⚠️ Warning, model is being initialized from an old checkpoint")
    
    checkpoint = torch.load(
        "learning/old_models/stateflow_winner_action.pt",
        map_location=torch.device('cpu')
    )
    model_state = checkpoint["model_state"]

    # update old model state to match the new one
    mismatch = action_value_model.load_state_dict(model_state, strict=False)
    model_state = action_value_model.state_dict()

    missing_keys, unexpected_keys = mismatch

    for mk in missing_keys:
        print("missing key", mk)

    for uk in unexpected_keys:
        print("unexpected key", uk)

⚠️⚠️⚠️ Warning, model is being initialized from an old checkpoint
missing key attack_value_fn.edge_value_fn.weight
missing key attack_value_fn.edge_value_fn.bias
missing key end_turn_value_fn.attention_coef_fn.transform_fn.weight
missing key end_turn_value_fn.summary_fn.weight
missing key end_turn_value_fn.summary_fn.bias
unexpected key winner_logit_fn.attention_coef_fn.transform_fn.weight
unexpected key winner_logit_fn.summary_fn.weight
unexpected key winner_logit_fn.summary_fn.bias
unexpected key attack_logit_fn.edge_value_fn.weight
unexpected key attack_logit_fn.edge_value_fn.bias
unexpected key end_turn_logit_fn.attention_coef_fn.transform_fn.weight
unexpected key end_turn_logit_fn.summary_fn.weight
unexpected key end_turn_logit_fn.summary_fn.bias


In [10]:
# Load the model before everything else so that I can freeze some layers
if model_state is not None:
    action_value_model.load_state_dict(model_state)

action_value_model.gat_layers[0].requires_grad_(False)
# action_value_model.gat_layers[1].requires_grad_(False)
# action_value_model.gat_layers[2].requires_grad_(False)

action_value_model.gat_layers[1].reset_parameters() 
# nn.init.constant_(action_value_model.gat_layers[1].passthrough_coef, -1)

action_value_model.gat_layers[2].reset_parameters() 
# nn.init.constant_(action_value_model.gat_layers[2].passthrough_coef, -1)

In [11]:
# ── CSV set‑up: log one row per epoch ───────────────────────────────────────
csv_path = f"{model_name}_epoch_metrics.csv"
epoch_log_file_exists = os.path.isfile(csv_path)
if epoch_log_file_exists and os.path.getsize(csv_path) == 0:
    os.remove(csv_path)
    epoch_log_file_exists = False

csv_file = open(csv_path, "a", newline="")  # Open in append mode
csv_writer = csv.writer(csv_file)

# Only write header if file doesn't exist yet
if not epoch_log_file_exists:
    csv_writer.writerow([
        "epoch",           # 0‑based epoch index
        "train_loss",      # average training loss for the epoch
        "val_loss",        # average validation loss for the epoch
        "train_time_sec",  # seconds spent in training phase
        "val_time_sec",    # seconds spent in validation phase
        "total_time_sec"   # train + val
    ])

# ── model / optimizer prep ─────────────────────────────────────────────────

n_epochs   = 1000
device     = torch.device("cuda", 0)
train_time_limit = 10 * 60
val_time_limit = train_time_limit / 5
criterion  = nn.MSELoss()

action_value_model = action_value_model.to(device)

optimizer  = torch.optim.Adam(
    [p for p in action_value_model.parameters() if p.requires_grad],
    lr=1e-2
)
reset_optimizer = True
if not reset_optimizer and optim_state is not None:
    optimizer.load_state_dict(optim_state)

scheduler  = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min",
    factor=0.5,
    patience=5,
    threshold=0.0001,
    min_lr=1e-8
)


In [12]:
# ── INITIAL (untrained) LOSS EVALUATION ───────────────────────────────────────
if not epoch_log_file_exists:
    action_value_model.eval()
    with torch.no_grad():
        t_start = time.time()

        # avg loss on training set *without* gradient tracking
        init_train_sum, init_train_batches = 0.0, 0
        for b in tqdm.tqdm(train_loader, desc="init‑train"):
            b = move_to_device(b, device)
            out = action_value_model(*extract_input(b))
            o, t = extract_output_target(b, out, mean_val, std_val)
            init_train_sum += criterion(o, t).item()
            init_train_batches += 1
            if (time.time() - t_start) > train_time_limit: 
                print(f"Stopping training after >{train_time_limit} seconds.")
                break
        init_train_loss = init_train_sum / init_train_batches
        t_train_done = time.time()

        # avg loss on validation set
        init_val_sum, init_val_batches = 0.0, 0
        for vb in tqdm.tqdm(val_loader, desc="init‑val"):
            vb = move_to_device(vb, device)
            vout = action_value_model(*extract_input(vb))
            vo, vt = extract_output_target(
                vb, vout, mean_val, std_val
            )
            init_val_sum += criterion(vo, vt).item()
            init_val_batches += 1
            if (time.time() - t_train_done) > val_time_limit:
                print(f"Stopping validation after >{val_time_limit} seconds.")
                break
        init_val_loss = init_val_sum / init_val_batches
        t_val_done = time.time()

    # times
    init_train_time = t_train_done - t_start
    init_val_time   = t_val_done - t_train_done
    init_total_time = t_val_done - t_start

    # write initial row (epoch = None)
    csv_writer.writerow([
        -1,
        init_train_loss,
        init_val_loss,
        init_train_time,
        init_val_time,
        init_total_time
    ])
    csv_file.flush()
    print(
        f"Initial (-1) | "
        f"Train {init_train_loss:.4f} | "
        f"Val {init_val_loss:.4f} | "
        f"Time {init_total_time:.1f}s (T {init_train_time:.1f}s | "
        f"V {init_val_time:.1f}s)"
    )

init‑val: 100%|██████████| 3/3 [00:21<00:00,  7.31s/it]

Initial (-1) | Train 1.7770 | Val 1.7762 | Time 192.8s (T 170.9s | V 21.9s)


In [13]:
scheduler.get_last_lr()

[0.01]

In [14]:
for epoch in range(start_epoch, n_epochs):
    t_start = time.time()

    # ── TRAINING -----------------------------------------------------------
    action_value_model.train()
    sum_train_loss = torch.tensor(0.0, device=device)
    n_train_batches = 0

    t_batch_tqdm = tqdm.tqdm(
        train_loader, 
        desc=f"train {epoch}" 
    )
    for t_batch in t_batch_tqdm:
        t_batch = move_to_device(t_batch, device)

        optimizer.zero_grad()

        model_out = action_value_model(*extract_input(t_batch))
        outputs, targets = extract_output_target(t_batch, model_out, mean_val, std_val)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
    
        if torch.isnan(loss) or torch.isinf(loss):
            raise ValueError(f"Loss contains NaN or Inf values")
        
        sum_train_loss += loss.detach()
        n_train_batches += 1
        t_batch_tqdm.set_postfix(av_loss=sum_train_loss.item()/n_train_batches)
        if (time.time() - t_start) > train_time_limit: 
            print(f"Stopping training after >{train_time_limit} seconds.")
            break
        
    t_batch_tqdm.close()

    avg_train_loss = sum_train_loss.item() / n_train_batches
    t_train_done = time.time()

    # ── VALIDATION ---------------------------------------------------------
    action_value_model.eval()
    sum_val_loss = 0.0
    n_val_batches = 0
    with torch.no_grad():
        v_batch_tqdm = tqdm.tqdm(
            val_loader, 
            desc=f"val {epoch}" 
        )
        for v_batch in v_batch_tqdm:
            v_batch = move_to_device(v_batch, device)

            v_out = action_value_model(*extract_input(v_batch))
            v_outputs, v_targets = extract_output_target(v_batch, v_out, mean_val, std_val)
            val_loss = criterion(v_outputs, v_targets)

            if torch.isnan(val_loss) or torch.isinf(val_loss):
                raise ValueError(f"Loss contains NaN or Inf values")
            
            sum_val_loss += val_loss
            n_val_batches += 1
            v_batch_tqdm.set_postfix(av_loss=sum_val_loss.item()/n_val_batches)
            if (time.time() - t_train_done) > val_time_limit:
                print(f"Stopping validation after >{val_time_limit} seconds.")
                break

        v_batch_tqdm.close()

    avg_val_loss = sum_val_loss.item() / n_val_batches
    t_val_done = time.time()

    # Step the learning rate scheduler after each epoch
    scheduler.step(avg_val_loss)

    # ── CSV logging ---------------------------------------------------------
    train_time = t_train_done - t_start
    val_time   = t_val_done   - t_train_done
    total_time = t_val_done   - t_start


    checkpoint_data = {
        "epoch": epoch,
        "optim_state": optimizer.state_dict(),
        "model_state": action_value_model.state_dict()
    }
    torch.save(checkpoint_data, os.path.join(models_dir, f"{model_name}_{epoch:06}.pt"))
    
    csv_writer.writerow([
        epoch,
        avg_train_loss,
        avg_val_loss,
        train_time,
        val_time,
        total_time
    ])
    csv_file.flush()  # ensure data is written even if run aborts

    # ── console printout ----------------------------------------------------
    print(
        f"Epoch {epoch} | "
        f"Train {avg_train_loss:.4f} | "
        f"Val {avg_val_loss:.4f} | "
        f"LR {scheduler.get_last_lr()[0]:.6f}"
    )
# ── tidy‑up ----------------------------------------------------------------
csv_file.close()

val 0: 100%|██████████| 3/3 [00:22<00:00,  7.55s/it, av_loss=1.06]


Epoch 0 | Train 3.4065 | Val 1.0633 | LR 0.010000


val 1: 100%|██████████| 3/3 [00:22<00:00,  7.52s/it, av_loss=0.998]


Epoch 1 | Train 1.0781 | Val 0.9980 | LR 0.010000


val 2: 100%|██████████| 3/3 [00:22<00:00,  7.47s/it, av_loss=0.973]


...

Epoch 89 | Train 0.9317 | Val 0.9013 | LR 0.000005


val 90: 100%|██████████| 3/3 [00:25<00:00,  8.46s/it, av_loss=0.911]


Epoch 90 | Train 0.9328 | Val 0.9107 | LR 0.000005


train 91:   0%|          | 0/21 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
type(float(torch.tensor([1])))